# Exercícios do módulo de Obtenção e Preparação de Dados

<img src="https://www.qads.com.br/img/QADS.png" width="300" height="170">

Remis Balaniuk, PhD

#Bibliotecas

In [ ]:
import re
import os
import json
import zipfile
import requests
import pandas as pd
import urllib.request

from bs4 import BeautifulSoup
from google.colab import files
from unicodedata import normalize
from IPython.display import display

# Flat files

Obtenha os dados abertos das [viagens realizadas a serviço no Portal da Transparência](http://www.portaltransparencia.gov.br/download-de-dados/viagens), processe as colunas normalizando atributos textuais e convertendo os demais para os tipos corretos e ao final salve o(s) dataframe(s) em disco usando o Pickle.

In [ ]:
ano=2021
url='http://www.portaltransparencia.gov.br/download-de-dados/viagens/'+str(ano)
print(url)
file_zip=str(ano)+'.zip'

File = urllib.request.urlretrieve(url, file_zip)

http://www.portaltransparencia.gov.br/download-de-dados/viagens/2021


In [ ]:
zipfile.ZipFile(file_zip).extractall()

os.listdir()

['.config',
 '2021_Trecho.csv',
 '2021_Passagem.csv',
 '2021.zip',
 '2021_Pagamento.csv',
 '2021_Viagem.csv',
 'sample_data']

In [ ]:
!apt install file

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (401 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32

In [ ]:
#Para detectar a codificação de arquivos:
!file -i 2021_Viagem.csv

2021_Viagem.csv: text/plain; charset=iso-8859-1


In [ ]:
#'ISO-8859-1' --> 'latin1'
df = pd.read_csv('2021_Viagem.csv', encoding='latin1', sep=';')

display( df.head(2) )
print(df.shape)

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor outros gastos
0,16829271,000163/21,Realizada,SIM,A viagem estva programada para ser realizda em...,36000,Ministério da Saúde,36211,Fundação Nacional de Saúde,***.980.641-**,VILIDIANA MORAES MOURA,ENGENHEIRO,FCT-03,Função Comissionada Técnica,03/02/2021,04/02/2021,Nova Brasilândia/MT,Visita técnica no municipio de Nova Brasilãndi...,"223,86","0,00","0,00"
1,16889781,000120/20,Não realizada,NÃO,Sem informação,26000,Ministério da Educação,26281,Fundação Universidade Federal de Sergipe,***.157.645-**,ROSE MARY SILVEIRA MENEZES,ASSISTENTE EM ADMINISTRAçãO,FG-0001,FG-0001,11/05/2021,16/05/2021,Fortaleza/CE,Participação no XXI Congresso Nacional de Secr...,"0,00","1179,77","0,00"


(147510, 21)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147480 entries, 0 to 147479
Data columns (total 21 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Identificador do processo de viagem  147480 non-null  int64 
 1   Número da Proposta (PCDP)            147480 non-null  object
 2   Situação                             147480 non-null  object
 3   Viagem Urgente                       147480 non-null  object
 4   Justificativa Urgência Viagem        147473 non-null  object
 5   Código do órgão superior             147480 non-null  int64 
 6   Nome do órgão superior               147480 non-null  object
 7   Código órgão solicitante             147480 non-null  int64 
 8   Nome órgão solicitante               147480 non-null  object
 9   CPF viajante                         147457 non-null  object
 10  Nome                                 147480 non-null  object
 11  Cargo                     

In [ ]:
df['Identificador do processo de viagem'].nunique()

147480

In [ ]:
df.describe(include='O').T.drop(columns='count')

,unique,top,freq
Número da Proposta (PCDP),27899,Informações pro,39006
Situação,2,Realizada,142110
Viagem Urgente,2,SIM,81467
Justificativa Urgência Viagem,27565,Informação protegida por sigilo nos termos da ...,39006
Nome do órgão superior,19,Ministério da Justiça e Segurança Pública,39955
Nome órgão solicitante,174,Sem informação,26090
CPF viajante,52187,***.000.000-**,643
Nome,41228,Informações protegidas por sigilo,38983
Cargo,494,Informações protegidas por sigilo,39006
Função,135,-1,55304


In [ ]:
#Valores nulos absolutos:
a=df.isnull().sum().sort_values(ascending=False)
a[a!=0]

#Os valores nulos de 'Descrição Função' poderiam ser substituídos por 'Não Informado',
#já que esse valor é utilizada nessa coluna (como também, nas outras colunas).

Cargo                            63890
Descrição Função                    41
CPF viajante                        23
Justificativa Urgência Viagem        7
dtype: int64

In [ ]:
#Datas:
df['Período - Data de início'] = pd.to_datetime(df['Período - Data de início'], dayfirst=True, errors='raise')
df['Período - Data de fim']    = pd.to_datetime(df['Período - Data de fim'], dayfirst=True, errors='raise')

#Para voltar ao formato '%d/%m/%Y', porém com o dtype=='object':
#df['Período - Data de início'].dt.strftime('%d/%m/%Y')

In [ ]:
#Valores --> Floats:
df['Valor diárias']=df['Valor diárias'].map( lambda v: v.replace(',','.') ).astype('float64')
df['Valor passagens']=df['Valor passagens'].map( lambda v: v.replace(',','.') ).astype('float64')
df['Valor outros gastos']=df['Valor outros gastos'].map( lambda v: v.replace(',','.') ).astype('float64')

In [ ]:
df.dtypes

Identificador do processo de viagem             int64
Número da Proposta (PCDP)                      object
Situação                                       object
Viagem Urgente                                 object
Justificativa Urgência Viagem                  object
Código do órgão superior                        int64
Nome do órgão superior                         object
Código órgão solicitante                        int64
Nome órgão solicitante                         object
CPF viajante                                   object
Nome                                           object
Cargo                                          object
Função                                         object
Descrição Função                               object
Período - Data de início               datetime64[ns]
Período - Data de fim                  datetime64[ns]
Destinos                                       object
Motivo                                         object
Valor diárias               

In [ ]:
#Podemos apenas normalizar as colunas do tipo object e salvar o dataframe com a codificação utf-8:

for col in df.columns[df.dtypes=='object']:
  df[col]=df[col].str.normalize('NFKD')

df.to_csv('df_utf-8.csv', encoding='utf-8', sep=';')
display( pd.read_csv('df_utf-8.csv', sep=';').head(2) )

df.to_pickle('Dataframe_viagens_utf-8.pkl.bz2', compression='bz2')

,Unnamed: 0,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor outros gastos
0,0,16829271,000163/21,Realizada,SIM,A viagem estva programada para ser realizda em...,36000,Ministério da Saúde,36211,Fundação Nacional de Saúde,***.980.641-**,VILIDIANA MORAES MOURA,ENGENHEIRO,FCT-03,Função Comissionada Técnica,2021-02-03,2021-02-04,Nova Brasilândia/MT,Visita técnica no municipio de Nova Brasilãn...,223.86,0.00,0.0
1,1,16889781,000120/20,Não realizada,NÃO,Sem informação,26000,Ministério da Educação,26281,Fundação Universidade Federal de Sergipe,***.157.645-**,ROSE MARY SILVEIRA MENEZES,ASSISTENTE EM ADMINISTRAçãO,FG-0001,FG-0001,2021-05-11,2021-05-16,Fortaleza/CE,Participação no XXI Congresso Nacional de Se...,0.00,1179.77,0.0


In [ ]:
#Teste...
'NÃO'==normalize('NFC',df.loc[1,'Viagem Urgente']), 'NÃO'==normalize('NFKD',df.loc[1,'Viagem Urgente']), normalize('NFKD','NÃO')==normalize('NFKD',df.loc[1,'Viagem Urgente'])

(True, False, True)

In [ ]:
#Remover acentos:
for col in df.columns[df.dtypes=='object']:
  df[col]=df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
df.head(2)

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor outros gastos
0,16829271,000163/21,Realizada,SIM,A viagem estva programada para ser realizda em...,36000,Ministerio da Saude,36211,Fundacao Nacional de Saude,***.980.641-**,VILIDIANA MORAES MOURA,ENGENHEIRO,FCT-03,Funcao Comissionada Tecnica,2021-02-03,2021-02-04,Nova Brasilandia/MT,Visita tecnica no municipio de Nova Brasilandi...,223.86,0.00,0.0
1,16889781,000120/20,Nao realizada,NAO,Sem informacao,26000,Ministerio da Educacao,26281,Fundacao Universidade Federal de Sergipe,***.157.645-**,ROSE MARY SILVEIRA MENEZES,ASSISTENTE EM ADMINISTRAcaO,FG-0001,FG-0001,2021-05-11,2021-05-16,Fortaleza/CE,Participacao no XXI Congresso Nacional de Secr...,0.00,1179.77,0.0


In [ ]:
#df.to_pickle('Dataframe_viagens.pkl')
df.to_pickle('Dataframe_viagens.pkl', compression='bz2')

# Expressões regulares

Escreva as expressões regulares solicitadas. Cada exercício possui um conjunto de testes que deve ser completamente validado pela expressão regular. Cada teste possui registros onde expera-se um *match* da expressão regular, enquanto outros registros não devem apresentar nenhum *match*. Faça o exercício para que todos os registros do teste fiquem com a marca de correto (em verde). Para os exercícios, não utilize expressões regulares com mais de 80 caracteres.



Para esse exercício você precisará baixar o arquivo zipado "exerciciosRegex" disponibilizado na pasta QADS em (https://drive.google.com/drive/folders/1sUeC2VYx93ZXi3tKCJarUTVvq_7C2ECI?usp=sharing). Este arquivo contém um diretório chamado "exercicios" e um arquivo chamado re_jupyter.py.

In [ ]:
file_zip='exerciciosRegex.zip'

In [ ]:
#Importar arquivos #Método 1:
uploaded = files.upload()

Saving exerciciosRegex.zip to exerciciosRegex.zip


In [ ]:
#Importar arquivos #Método 2:
id='https://drive.google.com/file/d/1mKGrtHEJCfKQ5pHnrDEHYPFxUXeNcvld/'.split('/')[-2]
url='https://drive.google.com/uc?id='+id+'&export=download'
print(url)

if not os.path.exists(file_zip):
  File = urllib.request.urlretrieve(url, file_zip)

https://drive.google.com/uc?id=1mKGrtHEJCfKQ5pHnrDEHYPFxUXeNcvld&export=download


In [ ]:
zipfile.ZipFile(file_zip).extractall()

In [ ]:
!cat re_jupyter.py | head -n20

import re
import os
import json
import itertools
from pygments import highlight
from pygments.lexers import XmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import display, HTML


certo = "url('')"

errado = "url('')"


def formatar_html(texto):
    return texto.replace(' ', '&nbsp;')

def formatar_match(texto, match):
    if match:


In [ ]:
!ls exercicios/

10.in	12.out	15.in	17.out	1.in	21.out	3.in   5.out  8.in
10.out	13.in	15.out	18.in	1.out	22.in	3.out  6.in   8.out
11.in	13.out	16.in	18.out	20.in	22.out	4.in   6.out  9.in
11.out	14.in	16.out	19.in	20.out	2.in	4.out  7.in   9.out
12.in	14.out	17.in	19.out	21.in	2.out	5.in   7.out


In [ ]:
#import imp
#import re_jupyter
#imp.reload(re_jupyter)

from re_jupyter import exercicio

--> Uma letra `a` imediatamente seguida por uma ou mais letras `b`

In [ ]:
exercicio(1, r'ab+')

,ab
,abb
,abbbbbbbbb
,cbaaabb
,aba
,a (sem ocorrências)
,baa (sem ocorrências)
,ba (sem ocorrências)
,cba (sem ocorrências)
,cbaaa (sem ocorrências)
,ba (sem ocorrências)


--> Uma palavra de 6 letras que inicia com a letra `z`

In [ ]:
re.findall('\Az\w{5}\Z', 'zangas')

['zangas']

In [ ]:
#exercicio(2, r'^z\w{5}$')
#exercicio(2, r'\Az\w{5}\Z')
exercicio(2, r'\bz\w{5}\b')

,zangas
,zangas espaço
,zéfiro
,segundo zéfiro
,zelo (sem ocorrências)
,zelos (sem ocorrências)
,zangada (sem ocorrências)
,zangados (sem ocorrências)
,azul (sem ocorrências)
,aziago (sem ocorrências)
,azougue (sem ocorrências)


--> Uma palavra com 8 a 10 letras que termine com a letra 'z'

In [ ]:
#exercicio(3, r'\b\w{8,10}\b') #Funciona para estes teste...
exercicio(3, r'\b\w{7,9}z\b')

,robustez
,espaço robustez
,chafariz
,antes chafariz depois
,vetustez
,placidez
,insensatez
,gravidez
,invalidez
,luz (sem ocorrências)
,talvez (sem ocorrências)


--> 3 dígitos consecutivos

In [ ]:
exercicio(4, r'\d{3}')

,040 inicia com zero
,056 de novo
,100 cem
,789 789 789
,0101 binário
,000x termina em x
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,vinte 20 (sem ocorrências)


--> Um número inteiro com 2, 3 ou 4 dígitos, sendo que o primeiro dígito não pode ser zero

In [ ]:
exercicio(5, r'\b([1-9]\d{1,3})\b')

,vinte 20
,100 cem
,789 789 789
,ano de 1984
,data 01/01/2018
,zero 0 (sem ocorrências)
,dígito 1 (sem ocorrências)
,inicia com zero 03 (sem ocorrências)
,03 também tem zero (sem ocorrências)
,040 inicia com zero (sem ocorrências)
,056 de novo (sem ocorrências)


--> Uma palavra que inicie com um dígito e termine com um letra

In [ ]:
#exercicio(6, r'\b(\d\w*[a-zA-Z])\b') # '1a' e '3a' são palavras que iniciam com um dígito e terminam com um letra.
exercicio(6, r'\b(\d\w+[a-zA-Z])\b')

,parágrafo 11b
,1111111b
,1bbbbbbb
,1b1b1b1b
,teste 1a (sem ocorrências)
,dígitos 113 total (sem ocorrências)
,soma quando 3a (sem ocorrências)
,bb1bbbbb11b (sem ocorrências)
,b1b1b1b1 (sem ocorrências)
,111 (sem ocorrências)
,1a1 (sem ocorrências)


--> Uma palavra seguida de um espaço seguido de um sequência de um ou mais dígitos

In [ ]:
exercicio(7, r'\w+ \d+')

,teste 1234
,parágrafo 12345
,letra 1
,a b 1 c
,cc 11 a
,1 1
,dois espaços não pode 2 (sem ocorrências)


--> Um dia da semana: segunda-feira, terça-feira, quarta-feira, quinta-feira, sexta-feira. Adicionalmente, permita a escrita dos dias da semana com espaço no lugar do hífen ou com as palavras unidas (ex.: quartafeira e quinta feira).

In [ ]:
#week=['segunda', 'terça', 'quarta', 'quinta', 'sexta', '']
#exercicio(8, r'\b('+'[ -]*feira|'.join(week)[::-1].replace('|','',1)[::-1]+')')

week=['segunda', 'terça', 'quarta', 'quinta', 'sexta']
exercicio(8, r'\b('+'|'.join([d+'[ -]*feira' for d in week])+')')

,hoje é segunda-feira
,amanhã será terça feira
,quartafeira já passou
,quinta-feira foi dia 3
,sextafeira
,sábado feriado (sem ocorrências)
,domingo feriado (sem ocorrências)
,2a feira internacional (sem ocorrências)
,sexta_feira (sem ocorrências)
,quinta.feira (sem ocorrências)
,domingues (sem ocorrências)


--> Uma URL no formato: http://www.&lt;dominio&gt;.com.br/&lt;arquivo&gt;.html. O &lt;arquivo&gt; e o &lt;domínio&gt; somente podem contar letras de a a z (minúscula ou maiúsculas), dígitos e os sinais _ e -.

In [ ]:
exercicio(9, r'http://www\.[\w-]+\.com\.br/[\w-]+\.html')

,http://www.google.com.br/index.html
,http://www.terra.com.br/underline_1.html
,http://www.teste-novo.com.br/hiFen-1.html
,http://www.google.com/index.html (sem ocorrências)
,http://uol.com.br/teste.html (sem ocorrências)
,http://bol.com.br/nao/podem/existir/barras.html (sem ocorrências)
,https://www.google.com/https/também/nao/pode.html (sem ocorrências)
,http://www.teste_velha.com.br/111-html (sem ocorrências)
,http://www-testevelho.com-br/111.html (sem ocorrências)
,http://www..com-br/1.html (sem ocorrências)
,http://www.com-br/2.html (sem ocorrências)


--> Duas palavras unidas pelas preposições 'de', 'do', 'da', 'dos' ou 'das'. As palavras somente podem conter letras e elas são separadas por um ou mais espaços.

In [ ]:
re.findall('d[aeo]', 'dae')
re.findall('d[aeo]\s', 'deo ')
re.findall('d[aeo]\s', 'de ')

['de ']

In [ ]:
#exercicio(10, r'[^\W]+\s+(de|do|da|dos|das)\s+[^\W]+')
#exercicio(10, r'\b([\w]+[\s]+d[aeo]s?[\s]+[\w]+)\b')  #pega o 'des' tbm...
#exercicio(10, r'\b([\w]+[\s]+(d(a|e|o)|d(a|o)s)[\s]+[\w]+)\b')
exercicio(10, r'\b(\w+\s+(de|d[ao]s?)\s+\w+)\b')

,"Olhos do mundo,"
,"zelos do marido,"
,"morte do Viegas,"
,uma vertigem de cabriolas
,descendente do cavalo de Aquiles
,a minha teoria das edições humanas
,me pareceu entrar na região dos gelos eternos
,"Esta idéia, (sem ocorrências)"
,"rútila e grande, (sem ocorrências)"
,"trajada ao bizarro, (sem ocorrências)"
,"como diria o Padre Bernardes, (sem ocorrências)"


--> Uma palavra de no mínimo 3 letras e com TODAS as letras maiúsculas

In [ ]:
#exercicio(11, r'\b([A-ZÁ-Ú]{3,})\b')
exercicio(11, r'\b([A-ZÁ-ÚÂ-Û]{3,})\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empreiteiras ODEBRECHT
,integrantes do Grupo OAS
,empresa GFD INVESTIMENTOS
,conforme já apurado pelo TCU
,"no caso das empresas MO Consultoria, Empreiteira Rigidez e RCI Software"
,RELATÓRIO DE TENDÊNCIAs


--> Duas palavras consecutivas, de no mínimo 2 letras cada, com TODAS as letras maiúsculas

In [ ]:
exercicio(12, r'\b([A-ZÁ-ÚÂ-Û]{2,}\s+[A-ZÁ-ÚÂ-Û]{2,})\b')
#exercicio(12, r'\b([A-ZÁ-ÚÂ-Û]{2,}\s+){1}[A-ZÁ-ÚÂ-Û]{2,}\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)
,integrantes do Grupo OAS (sem ocorrências)
,conforme já apurado pelo TCU (sem ocorrências)


,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)
,integrantes do Grupo OAS (sem ocorrências)
,conforme já apurado pelo TCU (sem ocorrências)


--> Duas ou mais palavras consecutivas com todas as letras maiúsculas. Cada palavra pode possuir qualquer quantidade de letras.

In [ ]:
#exercicio(13, r'\b([A-ZÁ-ÚÂ-Û]+\s*\b){2,}')    #falta 1
#exercicio(13, r'\b([A-ZÁ-ÚÂ-Û]+\b\s*\b){2,}')  #falta 1

exercicio(13, r'\b([A-ZÁ-ÚÂ-Û]+\s+){1,}[A-ZÁ-ÚÂ-Û]+\b')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,TESTE A TESTE
,A BE C DE
,VARIOS ESPACOS
,empreiteiras ODEBRECHT (sem ocorrências)


--> Uma linha contendo exatamente um endereço [IP](https://en.wikipedia.org/wiki/IP_address). ex.: 127.0.0.1, 192.168.0.254. Um IP é formado por exatamente 4 números separados por pontos. Cada número possui no máximo 3 dígitos.

In [ ]:
#exercicio(14, r'\b(\A\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$)\b')
#exercicio(14, r'\A\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$')
exercicio(14, r'\A(\d{1,3}\.){3}\d{1,3}$')

,127.0.0.1
,192.168.0.254
,10.11.10.11
,127.127.127.127
,9999.0.0.1 (sem ocorrências)
,127404041 (sem ocorrências)
,a127.0.0.1 (sem ocorrências)
,127.a.a.1 (sem ocorrências)
,127.0.0.1a (sem ocorrências)
,xxx.yyy.zzz.www (sem ocorrências)
,0.0.0.a (sem ocorrências)


--> Uma linha contendo exatamente um CPF com separador de milhares e hífen.

In [ ]:
#exercicio(15, r'\b(\d{3}\.){2}\d{3}-\d{2}\b')
exercicio(15, r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b')

,000.000.000-00
,111.111.111-11
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)
,111011011111 (sem ocorrências)
,11101101111 (sem ocorrências)
,1110110111 (sem ocorrências)


--> Uma linha contendo exatamente um CPF com separador de milhares opcional e hífen opcional.

In [ ]:
exercicio(16, r'\b\A(\d{3}[\.]?\d{3}[\.]?\d{3}-?\d{2}$)\b')

,000.000.000-00
,111.111.111-11
,111111.111-11
,111.111111-11
,11101101111
,0111.111.111-11 (sem ocorrências)
,111.111.111-110 (sem ocorrências)
,111.0111.0111-11 (sem ocorrências)
,11101110111-11 (sem ocorrências)
,111.111.111111 (sem ocorrências)
,1110110111111 (sem ocorrências)


--> Uma linha contendo exatamente um CNPJ com separador de milhares, barra e hífen.

In [ ]:
exercicio(17, r'\b\A(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$)\b')

,00.000.000/0001-00
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,00000.000/0001-00 (sem ocorrências)
,00000000/0001-00 (sem ocorrências)
,000000000001-00 (sem ocorrências)
,00000000000100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


--> Uma linha contendo exatamente um CNPJ com separador de milhares opcional, barra opcional e hífen opcional.

In [ ]:
exercicio(18, r'\b\A(\d{2}[\.]?\d{3}[\.]?\d{3}/?\d{4}-?\d{2}$)\b')

,00.000.000/0001-00
,00000.000/0001-00
,00000000/0001-00
,000000000001-00
,00000000000100
,xx.000.000/0001-xx (sem ocorrências)
,0011001000/0001-00 (sem ocorrências)
,00.000.00010001-00 (sem ocorrências)
,00.000.000/0001100 (sem ocorrências)
,x00000000000100 (sem ocorrências)
,00000000000100x (sem ocorrências)


--> Uma linha contendo exatamente um CEP com separador de milhares e hífen

In [ ]:
exercicio(19, r'\b\A(\d{2}\.\d{3}-\d{3}$)\b')

,70.000-000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,70000-000 (sem ocorrências)
,700000000 (sem ocorrências)
,70000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


--> Uma linha contendo exatamente um CEP com separador de milhares opcional e hífen opcional

In [ ]:
exercicio(20, r'\b\A(\d{2}[\.]?\d{3}-?\d{3}$)\b')

,70.000-000
,70000-000
,70000000
,7a.aaa-aaa (sem ocorrências)
,7000000000 (sem ocorrências)
,700000000 (sem ocorrências)
,7000000 (sem ocorrências)
,700000 (sem ocorrências)
,70000 (sem ocorrências)
,7000 (sem ocorrências)
,70000aaa (sem ocorrências)


--> Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx.

In [ ]:
#exercicio(21, r'\b\(\d{2}\)\d{4}-\d{4}') #errado?
#exercicio(21, r'\(\d{2}\)\d{4}-\d{4}$\b')
exercicio(21, r'^\(\d{2}\)\d{4}-\d{4}\b')

,(61)3316-0000
,3316-0000 (sem ocorrências)
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


--> Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx ou xxxx-xxxx (sem DDD).

In [ ]:
exercicio(22, r'^(\(\d{2}\))?\d{4}-\d{4}\b')

,(61)3316-0000
,3316-0000
,x61x3316x0000 (sem ocorrências)
,(61)3316x0000 (sem ocorrências)
,((61)3316-00006 (sem ocorrências)
,77(61)3316x000077 (sem ocorrências)
,(77)(61)3316-00007 (sem ocorrências)


# JSON

Usando a API do Banco Central, descrita em https://www.bcb.gov.br/conteudo/dadosabertos/BCBDepin/gnastportal-dados-abertostaxas-de-cambio---todos-os-boletins-diarios.pdf, escreva uma função que apresente ao usuário as moedas disponíveis para cotação, peça a ele que escolha uma ou mais, peça que o usuário escolha uma data inicial e uma data final, obtenha as cotações para as moedas selecionadas no período, armazene num dataframe e ao final salve o dataframe no disco usando o Pickle.

In [ ]:
#Lista de Moedas:
url='https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/Moedas?%24format=json'

In [ ]:
#Dicionário de moedas - Método 1 --> read_json():
df_moedas = pd.read_json(url)['value']
df_moedas

0    {'simbolo': 'AUD', 'nomeFormatado': 'Dólar aus...
1    {'simbolo': 'CAD', 'nomeFormatado': 'Dólar can...
2    {'simbolo': 'CHF', 'nomeFormatado': 'Franco su...
3    {'simbolo': 'DKK', 'nomeFormatado': 'Coroa din...
4    {'simbolo': 'EUR', 'nomeFormatado': 'Euro', 't...
5    {'simbolo': 'GBP', 'nomeFormatado': 'Libra Est...
6    {'simbolo': 'JPY', 'nomeFormatado': 'Iene', 't...
7    {'simbolo': 'NOK', 'nomeFormatado': 'Coroa nor...
8    {'simbolo': 'SEK', 'nomeFormatado': 'Coroa sue...
9    {'simbolo': 'USD', 'nomeFormatado': 'Dólar dos...
Name: value, dtype: object

In [ ]:
moedas = {df_moedas[i]['simbolo']:df_moedas[i]['nomeFormatado'] for i in range(df_moedas.shape[0])}
moedas

{'AUD': 'Dólar australiano',
 'CAD': 'Dólar canadense',
 'CHF': 'Franco suíço',
 'DKK': 'Coroa dinamarquesa',
 'EUR': 'Euro',
 'GBP': 'Libra Esterlina',
 'JPY': 'Iene',
 'NOK': 'Coroa norueguesa',
 'SEK': 'Coroa sueca',
 'USD': 'Dólar dos Estados Unidos'}

In [ ]:
#Dicionário de moedas - Método 2 --> request.get()+json.loads():
r = requests.get(url).content
data = json.loads(r)['value']
data

[{'nomeFormatado': 'Dólar australiano', 'simbolo': 'AUD', 'tipoMoeda': 'B'},
 {'nomeFormatado': 'Dólar canadense', 'simbolo': 'CAD', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Franco suíço', 'simbolo': 'CHF', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Coroa dinamarquesa', 'simbolo': 'DKK', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Euro', 'simbolo': 'EUR', 'tipoMoeda': 'B'},
 {'nomeFormatado': 'Libra Esterlina', 'simbolo': 'GBP', 'tipoMoeda': 'B'},
 {'nomeFormatado': 'Iene', 'simbolo': 'JPY', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Coroa norueguesa', 'simbolo': 'NOK', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Coroa sueca', 'simbolo': 'SEK', 'tipoMoeda': 'A'},
 {'nomeFormatado': 'Dólar dos Estados Unidos',
  'simbolo': 'USD',
  'tipoMoeda': 'A'}]

In [ ]:
moedas = {item['simbolo']:item['nomeFormatado'] for item in data}
moedas

{'AUD': 'Dólar australiano',
 'CAD': 'Dólar canadense',
 'CHF': 'Franco suíço',
 'DKK': 'Coroa dinamarquesa',
 'EUR': 'Euro',
 'GBP': 'Libra Esterlina',
 'JPY': 'Iene',
 'NOK': 'Coroa norueguesa',
 'SEK': 'Coroa sueca',
 'USD': 'Dólar dos Estados Unidos'}

_________

In [ ]:
def ListaMoedas():
  url='https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/Moedas?%24format=json'
  r = requests.get(url).content
  moedas = { item['simbolo']:item['nomeFormatado'] for item in json.loads(r)['value'] }
  #print(moedas)
  #display(moedas)
  for k in moedas.keys():
    print(f'\t{k}:{moedas[k]}')
  return moedas

In [ ]:
def CotacaoDolar():
  print('Moedas para cotação:')
  D = ListaMoedas()
  print()
  print('#(siglas separadas por espaço)')
  moedas = input('Moeda(s): ').split(' ')
  print(moedas)
  datainicial = input('Data inicial(mm-dd-aaaa): ')
  datafinal   = input('Data final(mm-dd-aaaa): ')
  aux = pd.DataFrame()
  for m in moedas:
    url=f'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?%40moeda=%27{m}%27&%40dataInicial=%27{datainicial}%27&%40dataFinalCotacao=%27{datafinal}%27&%24format=json'
    df = pd.json_normalize( pd.read_json(url)['value'] )
    df['moeda']=D[m]
    df['moeda_sigla']=m
    aux = pd.concat([aux, df], ignore_index=True)
  return aux

In [ ]:
bcb = CotacaoDolar()

Moedas para cotação:
	AUD:Dólar australiano
	CAD:Dólar canadense
	CHF:Franco suíço
	DKK:Coroa dinamarquesa
	EUR:Euro
	GBP:Libra Esterlina
	JPY:Iene
	NOK:Coroa norueguesa
	SEK:Coroa sueca
	USD:Dólar dos Estados Unidos

#(siglas separadas por espaço)
Moeda(s): AUD DKK JPY USD
['AUD', 'DKK', 'JPY', 'USD']
Data inicial(mm-dd-aaaa): 08-14-2021
Data final(mm-dd-aaaa): 08-17-2021


In [ ]:
bcb

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim,moeda,moeda_sigla
0,0.7325,0.7327,3.85340,3.85490,2021-08-16 10:02:22.187,Abertura,Dólar australiano,AUD
1,0.7322,0.7324,3.84360,3.84510,2021-08-16 11:08:17.306,Intermediário,Dólar australiano,AUD
2,0.7334,0.7335,3.84360,3.84460,2021-08-16 12:08:17.092,Intermediário,Dólar australiano,AUD
3,0.7338,0.7340,3.84850,3.85000,2021-08-16 13:05:49.693,Intermediário,Dólar australiano,AUD
4,0.7338,0.7340,3.85160,3.85310,2021-08-16 13:05:49.701,Fechamento,Dólar australiano,AUD
5,0.7280,0.7282,3.84100,3.84250,2021-08-17 10:04:56.608,Abertura,Dólar australiano,AUD
6,0.7267,0.7269,3.81410,3.81560,2021-08-17 11:06:19.052,Intermediário,Dólar australiano,AUD
7,0.7258,0.7260,3.81180,3.81320,2021-08-17 12:02:17.887,Intermediário,Dólar australiano,AUD
8,0.7253,0.7254,3.81150,3.81240,2021-08-17 13:10:47.971,Intermediário,Dólar australiano,AUD
9,0.7253,0.7254,3.81360,3.81450,2021-08-17 13:10:47.981,Fechamento,Dólar australiano,AUD


In [ ]:
bcb.to_pickle('Dataframe_Moedas.pkl', compression='bz2')

# HTML, JSON e REGEX

Crie um extrator de contratos a partir da seção 3 do Diário Oficial da União, obtendo as informações da página (link e texto) e por meio de expressões regulares extraia o nome do contratante, CNPJ e nome do contratado. Coloque o resultado num dataframe e salve em disco com o Pickle.

### Requisição

In [ ]:
#url1='http://www.imprensanacional.gov.br'
#secao='do3'; url2=f'https://www.in.gov.br/leiturajornal?data={data}&secao={secao}

secao='dou3'
data='09-08-2021'
url=f'https://www.in.gov.br/leiturajornal?secao={secao}&data={data}'
print(url)

https://www.in.gov.br/leiturajornal?secao=dou3&data=09-08-2021


In [ ]:
'''
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers=header)
'''

In [ ]:
%%time
#r=requests.get(url)
#soup=BeautifulSoup(r.text, 'html.parser')

r=requests.get(url).content
soup=BeautifulSoup(r, 'html.parser')

CPU times: user 149 ms, sys: 29.5 ms, total: 179 ms
Wall time: 17.8 s


In [ ]:
print(soup.prettify()[:600])

<!DOCTYPE html>
<html class="ltr" dir="ltr" lang="pt-BR">
 <head>
  <title>
   Leitura do Jornal - Imprensa Nacional
  </title>
  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
  <meta content="http://estruturaorganizacional.dados.gov.br/id/unidade-organizacional/332" property="creator.productor"/>
  <!-- FAVICON -->
  <link href="https://www.in.gov.br/o/imprensa-nacional-lf7_1-ce-theme/images/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="https://www.in.gov.br/o/imprensa-nacional-lf7_1-ce-theme/images/favicon//apple-icon-60x60.png" 


### Busca no HTML - Método 1:
--> Procurar por: ```<script id="params" type="application/json">```

In [ ]:
#Procurar por: <script id="params" type="application/json">
busca = soup.find_all('script', id="params")
print(busca[0].prettify()[:92])

<script id="params" type="application/json">
 {"typeNormDay":{"DO2ESP":false,"DO1ESP":false,


In [ ]:
D = json.loads( busca[0].text.replace('\n','').replace('\t','') )

### Busca no HTML - Método 2:
--> Procurar por: ```jsonArray```

In [ ]:
#Procurar por: jsonArray
busca2 = soup.find_all(text=re.compile('jsonArray'))

In [ ]:
type(busca2), len(busca2)

(bs4.element.ResultSet, 3)

In [ ]:
print(busca2[0][2:19])
print(busca2[1][4:100])
print(busca2[2][1:100])

	{"jsonArray":[]}
var elemParams = document.getElementById("_br_com_seatecnologia_in_buscadou_BuscaDouPortlet_para
	{"typeNormDay":{"DO2ESP":false,"DO1ESP":false,"DO1A":false,"DO3E":false,"DO2E":false,"DO1E":false}


In [ ]:
D2 = json.loads( busca2[2].replace('\n','').replace('\t','') )
D2.keys()

dict_keys(['typeNormDay', 'idPortletInstance', 'dateUrl', 'section', 'jsonArray'])

In [ ]:
D2==D

True

### Dicionário

In [ ]:
D.keys()

dict_keys(['typeNormDay', 'idPortletInstance', 'dateUrl', 'section', 'jsonArray'])

In [ ]:
type(D['jsonArray']), len(D['jsonArray'])

(list, 2680)

In [ ]:
D['jsonArray'][-1]

{'artType': 'Extrato de Termo Aditivo',
 'content': 'Extrato de Termo Aditivo Espécie: Termo de Rerratificação de Contrato - Tomada de Preços nº 02/2020 - CPL nº. 158/2020. Partes: Prefeitura de Sorocaba e a empresa GALLI INSTALAÇÕES E SERVIÇOS EIRELI - ME (***) - CNPJ: 18.787.702/0001-22. Objeto: retificação da clausula "1.2. Contrato de Repasse OGU 873908/2018 com Governo Federal através do Ministério do Esporte". Disponível no endereço: https://b...',
 'hierarchyLevelSize': 3,
 'hierarchyList': ['Prefeituras',
  'Estado de São Paulo',
  'Prefeitura Municipal de Sorocaba'],
 'hierarchyStr': 'Prefeituras/Estado de São Paulo/Prefeitura Municipal de Sorocaba',
 'numberPage': '249',
 'pubOrder': 'DO300051:00024:00710:00000:00000:00000:00000:00000:00000:00000:00055:00000',
 'title': 'Extrato de Termo Aditivo',
 'urlTitle': 'extrato-de-termo-aditivo-337230185'}

### DataFrame

In [ ]:
df_lista_url = pd.DataFrame(D['jsonArray'], columns=['title', 'urlTitle', 'content'])
df_lista_url.urlTitle = df_lista_url.urlTitle.apply(lambda s:'https://www.in.gov.br/web/dou/-/'+s)

display( df_lista_url.head(4) )
print( df_lista_url.shape )

,title,urlTitle,content
0,EXTRATO DE TERMO ADITIVO Nº 1.001/2019 - UASG ...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1.001/2019 - UASG ...
1,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...
2,EXTRATO DE TERMO APOSTILAMENTO,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO APOSTILAMENTO A repactuação d...
3,EXTRATO DE TERMO ADITIVO,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Espécie: Termo Aditiv...


(2680, 3)


### Regex - Extração de informações

In [ ]:
link = df_lista_url.urlTitle.loc[1]
print(link)

s=BeautifulSoup(requests.get(link).content, 'html.parser')

https://www.in.gov.br/web/dou/-/extrato-de-termo-aditivo-n-1/2021-uasg-130060-sfa/es/mapa-336997967


In [ ]:
#Método 1:
busca = s.find_all('p', class_="dou-paragraph")
busca

[<p class="dou-paragraph">Número do Contrato: 2/2020.</p>,
 <p class="dou-paragraph">Nº Processo: 21018.003713/2019-24.</p>,
 <p class="dou-paragraph">Pregão. Nº 6/2014. Contratante: SUPERINT.FED.DE AGRIC.,PEC.E ABASTECIMENTO/ES. Contratado: 27.355.635/0001-68 - MECANICA DO TREVO PECAS E ACESSORIOS LTDA - EPP. Objeto: O presente termo aditivo tem por objetivo a prorogar o prazo de vigencia de 15/06/2021 a 15/06/2022, do contrato original nº 02/2020. Vigência: 15/06/2020 a 15/06/2021. Valor Total Atualizado do Contrato: R$ 80.359,95. Data de Assinatura: 20/05/2021.</p>,
 <p class="dou-paragraph">(COMPRASNET 4.0 - 20/05/2021).</p>]

In [ ]:
L=[len(busca[i].text) for i in range(len(busca))]
#Argmax:
i = max(zip(L, range(len(L))))[1]
print(L,i)

texto = busca[i].text

[27, 34, 419, 30] 2


In [ ]:
#Método 2:
try:
  texto = s.find_all(text=re.compile('Contratante'))[0]
  print(texto)
except:
  print('String não encontrada.')

Pregão. Nº 6/2014. Contratante: SUPERINT.FED.DE AGRIC.,PEC.E ABASTECIMENTO/ES. Contratado: 27.355.635/0001-68 - MECANICA DO TREVO PECAS E ACESSORIOS LTDA - EPP. Objeto: O presente termo aditivo tem por objetivo a prorogar o prazo de vigencia de 15/06/2021 a 15/06/2022, do contrato original nº 02/2020. Vigência: 15/06/2020 a 15/06/2021. Valor Total Atualizado do Contrato: R$ 80.359,95. Data de Assinatura: 20/05/2021.


In [ ]:
Contratante = re.findall(r'Contratante: \b(([A-ZÀ-Û]+[\s\./,-]+){1,}[A-ZÀ-Û]+)\b', texto)[0][0]

#CNPJ = re.findall(r'\b(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\b', texto)[0]
CNPJ = re.findall(r'Contratado: \b(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\b', texto)[0]

#Contratado = re.findall(r'Contratado: '+CNPJ+' - (([A-ZÀ-Û]+[\s\./,-]+){1,})', texto)#[0][0]
Contratado = re.findall(r'Contratado: '+CNPJ+' - (([A-ZÀ-Û]+[\s\./,]+){1,}[A-ZÀ-Û]+)', texto)[0][0]

####################
print(f'Contratante: {Contratante}\nContratado:  {Contratado}\nCNPJ:        {CNPJ}')

Contratante: SUPERINT.FED.DE AGRIC.,PEC.E ABASTECIMENTO/ES
Contratado:  MECANICA DO TREVO PECAS E ACESSORIOS LTDA
CNPJ:        27.355.635/0001-68


In [ ]:
%%time
CNPJs=[]; #Empresas=[]
CNPJ_Contratado=[]; Contratado=[]; Contratante=[];
for link in df_lista_url.urlTitle:
  s = BeautifulSoup(requests.get(link).content, 'html.parser')
  busca = s.find_all('p', class_="dou-paragraph")
  L=[len(busca[i].text) for i in range(len(busca))]
  i = max(zip(L, range(len(L))))[1]  #Argmax:
  texto = busca[i].text
  try:
    #Empresas.append(re.findall(r'\b(([A-ZÀ-Û]+[\s\./,-]+){1,}[A-ZÀ-Û]+)\b', texto))
    CNPJs.append(re.findall(r'\b(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\b', texto))
    #print(CNPJs[-1])
    Contratante.append(re.findall(r'Contratante: \b(([A-ZÀ-Û]+[\s\./,-]+){1,}[A-ZÀ-Û]+)\b', texto)[0][0])
    try:
      CNPJ_Contratado.append(re.findall(r'Contratado: \b(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\b', texto)[0])
      Contratado.append(re.findall(r'Contratado: '+CNPJ_Contratado+' - (([A-ZÀ-Û]+[\s\./,]+){1,}[A-ZÀ-Û]+)', texto)[0][0])
    except:
      CNPJ_Contratado.append(None); Contratado.append(None);
  except:
    Contratante.append(None); CNPJ_Contratado.append(None); Contratado.append(None);

CPU times: user 2min 18s, sys: 4.03 s, total: 2min 22s
Wall time: 48min 29s


In [ ]:
'''
#Wall time: 1h 12s
%%time
CNPJ_Contratado=[]; Contratado=[]; Contratante=[];
for link in df_lista_url.urlTitle:
  s = BeautifulSoup(requests.get(link).content, 'html.parser')
  try:
    texto = s.find_all(text=re.compile('Contratante'))[0]
    Contratante.append(re.findall(r'Contratante: \b(([A-ZÀ-Û]+[\s\./,-]+){1,}[A-ZÀ-Û]+)\b', texto)[0][0])
    try:
      CNPJ_Contratado.append(re.findall(r'Contratado: \b(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})\b', texto)[0])
      Contratado.append(re.findall(r'Contratado: '+CNPJ_Contratado+' - (([A-ZÀ-Û]+[\s\./,]+){1,}[A-ZÀ-Û]+)', texto)[0][0])
    except:
      CNPJ_Contratado.append(''); Contratado.append('');
  except:
    Contratante.append(''); CNPJ_Contratado.append(''); Contratado.append('');
'''

In [ ]:
df_lista_url['CNPJs']=pd.Series(CNPJs)
df_lista_url['CNPJ_Contratado']=pd.Series(CNPJ_Contratado)
df_lista_url['Contratado']=pd.Series(Contratado)
df_lista_url['Contratante']=pd.Series(Contratante)

In [ ]:
#f=df_lista_url['CNPJ_Contratado'].notnull()
f=df_lista_url['Contratante'].notnull()
df_lista_url.loc[f]

,title,urlTitle,content,CNPJs,CNPJ_Contratado,Contratado,Contratante
0,EXTRATO DE TERMO ADITIVO Nº 1.001/2019 - UASG ...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1.001/2019 - UASG ...,[],None,None,INSTITUTO NACIONAL DE COLONIZACAO-E REFORMA AG...
1,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...,[27.355.635/0001-68],27.355.635/0001-68,None,"SUPERINT.FED.DE AGRIC.,PEC.E ABASTECIMENTO/ES"
6,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1300...,[27.355.635/0001-68],None,None,"SUPERINT.FED.DE AGRIC.,PEC.E ABASTECIMENTO/ES"
19,EXTRATO DE TERMO ADITIVO Nº 205/2021 - UASG 15...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 205/2021 - UASG 15...,[02.700.617/0001-45],None,None,CPST FED RN / CAMPUS CANGUARETAMA
22,EXTRATO DE TERMO ADITIVO Nº 3/2021 - UASG 1206...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 3/2021 - UASG 1206...,"[76.535.764/0001-43, 33.000.118/0001-79, 76.53...",None,None,GRUPAMENTO DE APOIO DE LAGOA SANTA
...,...,...,...,...,...,...,...
2506,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1600...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1600...,[09.087.150/0005-82],None,None,HOSPITAL MILITAR DE AREA DE BRASILIA
2510,EXTRATO DE TERMO ADITIVO Nº 6/2021 - UASG 1730...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 6/2021 - UASG 1730...,[09.370.244/0001-30],27.355.635/0001-68,None,SUPERINTENDENCIA DE SEGUROS PRIVADOS - RJ
2511,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1701...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 1/2021 - UASG 1701...,[13.515.708/0001-27],None,None,SUPERINTENDENCIA REGIONAL DA RFB NA
2533,EXTRATO DE TERMO ADITIVO Nº 5/2021 - UASG 1540...,https://www.in.gov.br/web/dou/-/extrato-de-ter...,EXTRATO DE TERMO ADITIVO Nº 5/2021 - UASG 1540...,[13.230.124/0001-05],None,None,FUNDACAO UNIVERSIDADE FEDERAL DO PIAUI


In [ ]:
df_lista_url.to_pickle('DO3.pkl', compression='bz2')

___________
___________